In [25]:
import rispy
from bs4 import BeautifulSoup
import requests, glob
import pandas as pd
import math

In [ ]:
df = pd.read_csv('merged_all_no_duplicates.csv')
df.set_index('DOI')

In [41]:
doi_abstract_dict = dict(zip(df['DOI'], df['Abstract Note']))

In [44]:
doi_abstract_dict['10.1109/ICComm.2018.8484771'.lower()]

KeyError: '10.1109/iccomm.2018.8484771'

In [36]:
def get_abstract_online(row):
        op,query = doi_url(row)

        if op == 1:
            doi = query
            article_url = f'https://link.springer.com/{doi}'
        elif op == 2:
            article_url = query

        # Send an HTTP GET request to the article URL
        try:
            response = requests.get(article_url)
        
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Parse the HTML content of the article page
                soup = BeautifulSoup(response.content, 'html.parser')
    
                # Find the abstract section using the HTML structure you provided
                abstract_section = soup.find('section', {'data-title': 'Abstract'})
    
                if abstract_section:
                    # Extract the text content of the abstract
                    abstract_text = abstract_section.find('div', {'class': 'c-article-section__content'}).get_text()
                    # Update the 'Abstract Note' column in the original DataFrame 'df'
                    row['abstract'] = abstract_text
                else:
                    row['abstract'] = "Abstract not found."
        except:
            row['abstract'] = "Failed to retrieve data."

In [38]:
def doi_url(row):
    query = ''
    try:
        query = row['doi']
        return 1,query
    except:
        try:
            query = row['urls']
            return 2,query
        except:
            query = 'https://link.springer.com/10.1007/s12083-020-00947-w'
            return 2,query

In [46]:
# Load RIS file
prefix = 'merge-all-'
input_files = glob.glob(f"ris/{prefix}*.ris")
merged_entries = []
survey_entries = []
for ris_file in input_files:
    print(f'Working on {ris_file}')
    with open(ris_file, 'r') as bibliography_file:
        entries = rispy.load(bibliography_file)
        for entry in entries:
            if 'abstract' not in entry:
                try:
                    abs = doi_abstract_dict[entry['doi']]
                except:
                    abs = float(0)
                if isinstance(abs,str):
                    entry['abstract'] = abs
                else:
                    get_abstract_online(entry)
            if 'survey' in entry['title'].lower():
                survey_entries.append(entry)
            else:
                if 'book' not in entry['type_of_reference']:
                    merged_entries.append(entry)

Working on ris/merge-all-13001-13500.ris
Working on ris/merge-all-14001-14500.ris
Working on ris/merge-all-3001-3500.ris
Working on ris/merge-all-0001-1000.ris
Working on ris/merge-all-7001-7500.ris
Working on ris/merge-all-7501-8000.ris
Working on ris/merge-all-failed.ris
Working on ris/merge-all-10501-11000.ris
Working on ris/merge-all-4001-4500.ris
Working on ris/merge-all-9501-10000.ris
Working on ris/merge-all-11501-12000.ris
Working on ris/merge-all-9001-9500.ris
Working on ris/merge-all-5501-6000.ris
Working on ris/merge-all-10001-10500.ris
Working on ris/merge-all-6001-6500.ris
Working on ris/merge-all-3501-4000.ris
Working on ris/merge-all-5001-5500.ris
Working on ris/merge-all-11001-11500.ris
Working on ris/merge-all-1001-1500.ris
Working on ris/merge-all-2001-2500.ris
Working on ris/merge-all-12001-12500.ris
Working on ris/merge-all-1501-2000.ris
Working on ris/merge-all-8501-9000.ris
Working on ris/merge-all-13501-14000.ris
Working on ris/merge-all-14501-14600.ris
Working o

In [47]:
with open('merged-final.ris', 'w') as bibliography_file:
    rispy.dump(merged_entries, bibliography_file)
with open('surveys-final.ris', 'w') as bibliography_file:
    rispy.dump(survey_entries, bibliography_file)